# Central Bank Policy Rates - BIS

## Python set-up

In [1]:
# system imports
from pathlib import Path
from urllib.error import HTTPError, URLError
from typing import Any
import textwrap as tw

In [2]:
# analytic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycountry

In [3]:
# local imports
from plotting import (
    set_chart_dir,
    line_plot,
    finalise_plot,
)

In [4]:
# plotting set-up
SOURCE = "Source: BIS policy rates"
LFOOTER = "Daily data.  Note: There are lags in BIS data reporting. "
plt.style.use("fivethirtyeight")
CHART_DIR = "./CHARTS/BIS/"
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)
set_chart_dir(CHART_DIR)
for filename in Path(CHART_DIR).glob("*.png"):
    filename.unlink()
SHOW = False

## Data capture

In [5]:
def country_code_to_name(country_code: str) -> str:
    """Convert 2-digit country codes to country names."""

    try:
        country = pycountry.countries.get(alpha_2=country_code)
        return country.name
    except AttributeError:
        if country_code == "XM":
            return "Euro Area"
        return country_code

In [6]:
def get_bis_data(start="2018-01-01") -> tuple[pd.DataFrame, str]:
    """Get central bank policy rates from the BIS for a select set of states.
    Arguments: start -- the start date for the data (default: 2018-01-01)
    Returns a DataFrame of daily data and a string of the latest rates."""

    states = sorted(list(set([  # ensure unique and sorted
        "AU", "CA", "GB", "JP", "NO", "KR", "NZ", "SE", "US", "XM",
        "AR", "BR", "CL", "CN", "CZ", "DK", "HK", "HU", "IN", "ID",
        "IL", "MY", "MX", "PH", "PL", "RU", "ZA", "TH", "TR", "CO",
        "HR", "IS", "MA", "MK", "PE", "RO", "SA", "RS", "CH",
    ])))
    box = {}
    finals = []
    for abbr in states:
        for _trys in range(2):
            state = country_code_to_name(abbr)
            print(state)
            url = f"https://stats.bis.org/api/v2/data/dataflow/BIS/WS_CBPOL/1.0/D.{abbr}?startPeriod={start}&format=csv"
            try:
                df = pd.read_csv(url)[["TIME_PERIOD", "OBS_VALUE"]]
            except (HTTPError, URLError) as e:
                print(f"Internet Error: {state} {e}")
                continue
            s = pd.Series(
                df["OBS_VALUE"].values,
                name=state,
                dtype=float,
                index=pd.to_datetime(df["TIME_PERIOD"]),
            )
            if s.empty or s.isnull().all():
                print(f"Empty: {state}")
                continue
            idx = pd.date_range(start=s.index.min(), end=s.index.max(), freq="D")
            s = s.reindex(idx, fill_value=np.nan)
            s = s.sort_index()
            s = s.ffill()
            box[state] = s
            finals.append(f"{abbr}: {s.iloc[-1]:.2f}")
            break

    data = pd.DataFrame(box)
    data.index = pd.PeriodIndex(data.index, freq="D")

    return data, ", ".join(finals)


df, latest_rates = get_bis_data()
print(tw.fill(latest_rates))

Argentina
Australia
Brazil
Canada
Switzerland
Chile
China
Colombia
Czechia
Denmark
United Kingdom
Hong Kong
Croatia
Hungary
Indonesia
Israel
India
Iceland
Japan
Korea, Republic of
Morocco
North Macedonia
Mexico
Malaysia
Norway
New Zealand
Peru
Philippines
Poland
Romania
Serbia
Russian Federation
Saudi Arabia
Sweden
Thailand
Türkiye
United States
Euro Area
South Africa
AR: 35.00, AU: 4.35, BR: 11.25, CA: 3.75, CH: 1.00, CL: 5.25, CN:
3.10, CO: 9.75, CZ: 4.00, DK: 2.85, GB: 4.75, HK: 5.00, HR: 0.00, HU:
6.50, ID: 6.00, IL: 4.50, IN: 6.50, IS: 8.50, JP: 0.25, KR: 3.25, MA:
2.75, MK: 5.80, MX: 10.25, MY: 3.00, NO: 4.50, NZ: 4.25, PE: 5.00, PH:
6.00, PL: 5.75, RO: 6.50, RS: 5.75, RU: 21.00, SA: 5.25, SE: 2.75, TH:
2.25, TR: 50.00, US: 4.62, XM: 3.25, ZA: 7.75


## Plotting

In [7]:
COUNTER = 0
def plot_rates(dataset: pd.DataFrame, start="2018-01-01") -> None:
    """Plot the central bank policy rates."""

    widths = [1.5] * 40
    if "Australia" in dataset.columns:
        widths[dataset.columns.get_loc("Australia")] = 3

    if len(dataset.columns) < 10:
        fs = 10
    elif len(dataset.columns) < 20: 
        fs = 8
    else:
        fs = 6

    global COUNTER
    line_plot(
        dataset[lambda x: x.index >= start],
        title="Central Bank Policy Rates",
        ylabel="Annual Policy Rate (%)",
        rfooter=SOURCE,
        lfooter=LFOOTER,
        width=widths,
        style=["-", "--", ":", "-."] * 10,
        legend={"ncols": 3, "loc": "upper left", "fontsize": fs},
        y0=True,
        zero_y=True,
        tags=f"{COUNTER}",
        show=SHOW,
    )
    COUNTER += 1


plot_rates(df[["Australia", "Canada", "Euro Area", "New Zealand", "United Kingdom", "United States"]])
ugly = ["Argentina", "Russian Federation", "Türkiye"]
plot_rates(df[sorted([x for x in df.columns if x not in ugly])])
plot_rates(df[ugly])

In [8]:
MEAN_MEDIAN = 0.80  # proportion of non-na data points to plot mean and median


def plot_world(
    data: pd.DataFrame,
    exclusions: None | list[str] = None,
    **kwargs: Any,
) -> None:
    """Plot Australia vs the BIS monitored mean and median."""

    # Exclude problematic BIS states
    keep = [x for x in data.columns if x not in exclusions]
    my_data = data[keep].copy()

    # plot remaining BIS states without legend label using the _ trick
    mapper = {x: f"_{x}" for x in my_data.columns}
    my_data = my_data.rename(columns=mapper)
    ax = my_data.plot(color="blue", lw=0.25, alpha=0.5)
    back = {y: x for x, y in mapper.items()}
    my_data = my_data.rename(columns=back)
    my_data["Australia"].dropna().plot(ax=ax, color="darkorange", lw=3, label="Australia")

    # plot mean if THRESHOLD proportion of non-na data points met
    mean = my_data.mean(axis=1).where(
        my_data.notna().sum(axis=1) >= len(my_data.columns) * MEAN_MEDIAN,
        other=np.nan,
    )
    median = my_data.median(axis=1).where(
        my_data.notna().sum(axis=1) >= len(my_data.columns) * MEAN_MEDIAN,
        other=np.nan,
    )
    mean.plot(ax=ax, color="darkblue", ls="--", lw=2, label="BIS monitored mean")
    median.plot(ax=ax, color="darkred", ls=":", lw=2, label="BIS monitored median")

    # plot
    global PW_COUNTER  # yes, this is ugly
    PW_COUNTER = PW_COUNTER + 1
    finalise_plot(
        ax,
        xlabel=None,
        y0=True,
        rfooter=SOURCE,
        tag=str(PW_COUNTER),
        legend={"loc": "best", "fontsize": "xx-small"},
        **kwargs,
        show=SHOW,
    )


PW_COUNTER = 0
plot_world(
    df, 
    exclusions=ugly,
    title="CB Policy Rates: Australia in World Context",
    ylabel="Annual Policy Rate (%)",
    lfooter=LFOOTER + f" Excluded: {', '.join(ugly)}.",
)

/Users/bryanpalmer/ABS/notebooks/plotting.py:344: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  fig.tight_layout(pad=1.1)


## The End

In [9]:
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Dec 18 2024 10:46:12

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.30.0

matplotlib: 3.10.0
pycountry : 24.6.1
pandas    : 2.2.3
numpy     : 1.26.4

Watermark: 2.5.0

